### 0. Import Libs

In [8]:
import pandas as pd 
import os 
import constant as c
import sys
import shutil
from zipfile import ZipFile
sys.path.append('../')
from util.get_coord_vw import GeoCoderVworld

### 1. Get Energy(Electricity and Gas) Data

In [9]:
def filter_df(df, year, month, sigungucds):
    cond1 = df['stdryear'] == year 
    cond2 = df['stdrmonth'] == month 
    cond3 = df['sigungucd'].isin(sigungucds)
    return df[cond1 & cond2 & cond3]

#### Gas

In [12]:
for yyyy in ['2020', '2021', '2022']:
    bdenergy_path = os.path.join(c.SSD_PATH, 'elctgas')
    before_txtfile_name_1 = f'mart_key_02_{yyyy}.txt'
    before_txtfile_name_2 = 'mart_key_02_YYYY.txt'
    after_txtfile_name = f'bdgas_{yyyy}.txt' 
    picklefile_name = f'bdgas_{yyyy}.p' 
    zipfile_name = f'국토교통부_건물에너지_가스에너지+({yyyy}년).zip'
    zipfile_path = os.path.join(bdenergy_path, zipfile_name) 
    txtfile_path = os.path.join(bdenergy_path, after_txtfile_name) 

    if os.path.exists(txtfile_path): 
        pass
    else:
        with ZipFile(zipfile_path, 'r') as zip_ref:
            zip_ref.extractall(bdenergy_path)
            if int(yyyy) >= 2020:
                try: 
                    os.rename(
                        os.path.join(bdenergy_path, 'MART_KEY_02', before_txtfile_name_1),
                        os.path.join(bdenergy_path, after_txtfile_name)
                        )
                except:
                    os.rename(
                        os.path.join(bdenergy_path, 'MART_KEY_02', before_txtfile_name_2),
                        os.path.join(bdenergy_path, after_txtfile_name)
                        )
                # shutil.rmtree(os.path.join(bdenergy_path, 'MART_KEY_02'))           
            else:
                os.rename(
                    os.path.join(bdenergy_path, before_txtfile_name_1),
                    os.path.join(bdenergy_path, after_txtfile_name)
                    )               
    bdgas = pd.read_csv(
            txtfile_path,
            sep= '|',
            encoding= 'CP949',
            header= None,
            dtype= 'string'
            )
    bdgas = bdgas.drop(bdgas.columns[-1], axis=1)
    bdgas.columns = c.BDGAS['gen_col_kr']
    bdgas['시도코드'] = [code[0:2] for code in bdgas['시군구코드']] 

    # Erase Data that lacks quality and preprocess
    to_erase_indexes = bdgas[bdgas['대지구분코드'].isna()].index # if 대지구분코드 is null -> deduct from dataset
    bdgas = bdgas.drop(index = to_erase_indexes)
    bdgas['stdryear'] = [code[0:4] for code in bdgas['사용년월']] 
    bdgas['stdrmonth'] = [code[4:6] for code in bdgas['사용년월']] 
    bdgas['pnu'] = bdgas['시군구코드'] + \
                        bdgas['법정동코드'] + \
                        ['1' if code == '0' else '2' if code == '1' else '0' for code in bdgas['대지구분코드']] + \
                        bdgas['번'] + bdgas['지']
    bdgas.replace({pd._libs.missing.NAType(): None}, inplace=True)
    bdgas = bdgas.replace({'\\\\': ''}, regex = True)
    bdgas = bdgas.drop_duplicates(keep='last')
    bdgas.columns = list(c.BDGAS['gen_col_types'].keys())
    bdgas = filter_df(bdgas, yyyy, '01', c.CBD_CDS)
    bdgas.to_csv(f'../asset/preprocess/features/df_bdgas_{yyyy}.csv', index = False) 

#### Electricity

In [14]:
# for yyyy in ['2020', '2021', '2022']:
for yyyy in ['2021', '2022']:
    bdenergy_path = os.path.join(c.SSD_PATH, 'elctgas')
    before_txtfile_name_1 = f'mart_key_01_{yyyy}.txt'
    before_txtfile_name_2 = 'mart_key_01_YYYY.txt'
    after_txtfile_name = f'bdelct_{yyyy}.txt' 
    picklefile_name = f'bdelct_{yyyy}.p' 
    zipfile_name = f'국토교통부_건물에너지_전기에너지+({yyyy}년).zip'
    zipfile_path = os.path.join(bdenergy_path, zipfile_name) 
    txtfile_path = os.path.join(bdenergy_path, after_txtfile_name) 

    if os.path.exists(txtfile_path): 
        pass

    else:
        with ZipFile(zipfile_path, 'r') as zip_ref:
            zip_ref.extractall(bdenergy_path)
            if int(yyyy) >= 2020:
                try: 
                    os.rename(
                        os.path.join(bdenergy_path, 'MART_KEY_01', before_txtfile_name_1),
                        os.path.join(bdenergy_path, after_txtfile_name)
                        )
                except:
                    os.rename(
                        os.path.join(bdenergy_path, 'MART_KEY_01', before_txtfile_name_2),
                        os.path.join(bdenergy_path, after_txtfile_name)
                        )
                # shutil.rmtree(os.path.join(bdenergy_path, 'MART_KEY_02'))           
            else:
                os.rename(
                    os.path.join(bdenergy_path, before_txtfile_name_1),
                    os.path.join(bdenergy_path, after_txtfile_name)
                    )               
    bdelct = pd.read_csv(
            txtfile_path,
            sep= '|',
            encoding= 'CP949',
            header= None,
            dtype= 'string'
            )
    bdelct = bdelct.drop(bdelct.columns[-1], axis=1)
    bdelct.columns = c.BDELCT['gen_col_kr']
    bdelct['시도코드'] = [code[0:2] for code in bdelct['시군구코드']] 

    # Erase Data that lacks quality and preprocess
    to_erase_indexes = bdelct[bdelct['대지구분코드'].isna()].index # if 대지구분코드 is null -> deduct from dataset
    bdelct = bdelct.drop(index = to_erase_indexes)
    bdelct['stdryear'] = [code[0:4] for code in bdelct['사용년월']] 
    bdelct['stdrmonth'] = [code[4:6] for code in bdelct['사용년월']] 
    bdelct['pnu'] = bdelct['시군구코드'] + \
                        bdelct['법정동코드'] + \
                        ['1' if code == '0' else '2' if code == '1' else '0' for code in bdelct['대지구분코드']] + \
                        bdelct['번'] + bdelct['지']
    bdelct.replace({pd._libs.missing.NAType(): None}, inplace=True)
    bdelct = bdelct.replace({'\\\\': ''}, regex = True)
    bdelct = bdelct.drop_duplicates(keep='last')
    bdelct.columns = list(c.BDELCT['gen_col_types'].keys())
    bdelct = filter_df(bdelct, yyyy, '01', c.CBD_CDS)
    bdelct.to_csv(f'../asset/preprocess/features/df_bdelct_{yyyy}.csv', index = False) 

In [21]:
bdgas2020 = pd.read_csv('../asset/preprocess/features/df_bdgas_2020.csv')
bdgas2021 = pd.read_csv('../asset/preprocess/features/df_bdgas_2021.csv')
bdgas2022 = pd.read_csv('../asset/preprocess/features/df_bdgas_2022.csv')
bdgas = pd.concat([bdgas2020, bdgas2021, bdgas2022], ignore_index=True)
bdgas.to_csv(f'../asset/preprocess/features/df_bdgas.csv', index = False) 
bdgas.head(3)

,useym,platplc,newplatplc,sigungucd,bjdongcd,platgbcd,bun,ji,naroadnum,naroadcd,naugrndcd,namainbun,nasubbun,kwh,rlgcd,stdryear,stdrmonth,pnu
0,202001,서울특별시 중구 신당동 102-5번지,서울특별시 중구 다산로 232,11140,16200,0,102,5,1,1.114031e+11,0.0,232.0,0.0,23239,11,2020,1,1114016200101020005
1,202001,서울특별시 중구 신당동 102-7번지,서울특별시 중구 다산로38길 5,11140,16200,0,102,7,1,1.114041e+11,0.0,5.0,0.0,1304,11,2020,1,1114016200101020007
2,202001,서울특별시 중구 신당동 103-1번지,서울특별시 중구 다산로36길 5,11140,16200,0,103,1,1,1.114041e+11,0.0,5.0,0.0,22717,11,2020,1,1114016200101030001


In [22]:
# Read df_bdgas
bdgas = pd.read_csv('../asset/preprocess/features/df_bdgas.csv')
bdgas.head(3)

,useym,platplc,newplatplc,sigungucd,bjdongcd,platgbcd,bun,ji,naroadnum,naroadcd,naugrndcd,namainbun,nasubbun,kwh,rlgcd,stdryear,stdrmonth,pnu
0,202001,서울특별시 중구 신당동 102-5번지,서울특별시 중구 다산로 232,11140,16200,0,102,5,1,1.114031e+11,0.0,232.0,0.0,23239,11,2020,1,1114016200101020005
1,202001,서울특별시 중구 신당동 102-7번지,서울특별시 중구 다산로38길 5,11140,16200,0,102,7,1,1.114041e+11,0.0,5.0,0.0,1304,11,2020,1,1114016200101020007
2,202001,서울특별시 중구 신당동 103-1번지,서울특별시 중구 다산로36길 5,11140,16200,0,103,1,1,1.114041e+11,0.0,5.0,0.0,22717,11,2020,1,1114016200101030001


In [23]:
bdelct2020 = pd.read_csv('../asset/preprocess/features/df_bdelct_2020.csv')
bdelct2021 = pd.read_csv('../asset/preprocess/features/df_bdelct_2021.csv')
bdelct2022 = pd.read_csv('../asset/preprocess/features/df_bdelct_2022.csv')
bdelct = pd.concat([bdelct2020, bdelct2021, bdelct2022], ignore_index=True)
bdelct.to_csv(f'../asset/preprocess/features/df_bdelct.csv', index = False) 
bdelct.head(3)

,useym,platplc,newplatplc,sigungucd,bjdongcd,platgbcd,bun,ji,naroadnum,naroadcd,naugrndcd,namainbun,nasubbun,kwh,rlgcd,stdryear,stdrmonth,pnu
0,202001,서울특별시 영등포구 여의도동 15-13번지,서울특별시 영등포구 국회대로68길 24,11560,11000,0,15,13,1,1.156042e+11,0.0,24.0,0.0,64987,11,2020,1,1156011000100150013
1,202001,서울특별시 영등포구 여의도동 15-12번지,서울특별시 영등포구 국회대로66길 17,11560,11000,0,15,12,1,1.156042e+11,0.0,17.0,0.0,52993,11,2020,1,1156011000100150012
2,202001,서울특별시 영등포구 여의도동 15-11번지,서울특별시 영등포구 국회대로68길 18,11560,11000,0,15,11,1,1.156042e+11,0.0,18.0,0.0,86400,11,2020,1,1156011000100150011


In [24]:
# Read df_bdelct
bdelct = pd.read_csv('../asset/preprocess/features/df_bdelct.csv')
bdelct.head(3)

,useym,platplc,newplatplc,sigungucd,bjdongcd,platgbcd,bun,ji,naroadnum,naroadcd,naugrndcd,namainbun,nasubbun,kwh,rlgcd,stdryear,stdrmonth,pnu
0,202001,서울특별시 영등포구 여의도동 15-13번지,서울특별시 영등포구 국회대로68길 24,11560,11000,0,15,13,1,1.156042e+11,0.0,24.0,0.0,64987,11,2020,1,1156011000100150013
1,202001,서울특별시 영등포구 여의도동 15-12번지,서울특별시 영등포구 국회대로66길 17,11560,11000,0,15,12,1,1.156042e+11,0.0,17.0,0.0,52993,11,2020,1,1156011000100150012
2,202001,서울특별시 영등포구 여의도동 15-11번지,서울특별시 영등포구 국회대로68길 18,11560,11000,0,15,11,1,1.156042e+11,0.0,18.0,0.0,86400,11,2020,1,1156011000100150011
